In [4]:
from pynq import Device, Overlay
ol = Overlay("bitstreams/u280_xclbin/500_250_HBM/FullW.xclbin",download=True,device=Device.devices[0])
print("Overlay downloaded.")


Overlay downloaded.


In [2]:
ol

In [5]:
ol

In [15]:
ol.timestamp

'2022/4/29 15:28:20 +284494'

In [6]:
from pynq import allocate 
import numpy as np

A = allocate(shape=(500,250), dtype=np.complex64, target=getattr(ol,"HBM0"))

In [14]:
cu = 0
resolution = 500
gridsize = 250
ROW=500
COL=250

def init_hw_dotProduct(s_in_V,s2_in_V):       
    COL = 250
    ROW = 500
    for col in range(COL):
        s2_in_V[col] = col * 1.0
        for row in range(ROW):
            s_in_V[row][col] = complex(row*1.0,row*col*0.33)
            
    return s_in_V, s2_in_V

def init_hw_updateDirection(resV_V,kappa_V):       
    for row in range(ROW):
        resV_V[row] = complex(row*1.0,row*0.33)
        for col in range(COL):
            kappa_V[row][col] = complex(row*1.0,row*col*0.33)
    return resV_V, kappa_V

dotprod = ol.dotprod_1
update = ol.update_1

B = allocate(shape=(gridsize,), dtype=np.float32, target=getattr(ol,"HBM1"))
C = allocate(shape=(resolution,), dtype=np.complex64, target=getattr(ol,"HBM2"))

D = allocate(shape=(resolution,gridsize), dtype=np.complex64,  target=getattr(ol,"HBM6"))
E = allocate(shape=(resolution,),dtype=np.complex64,  target=getattr(ol,"HBM7"))
F = allocate(shape=(gridsize), dtype=np.complex64, target=getattr(ol,"HBM8"))

   
A, B = init_hw_dotProduct(A, B)
E, D = init_hw_updateDirection(E,D)

print(f'start dotprod_{cu}')
A.sync_to_device()
B.sync_to_device()
dotprod.call(A,B,C)   
C.sync_from_device()
print(f'end dotprod_{cu}')

print(f'start update_{cu}')
D.sync_to_device()
E.sync_to_device()
update.call(D,E,F)
F.sync_from_device()
print(f'end update_{cu}')



start dotprod_0
end dotprod_0
start update_0
end update_0
